In [1]:
#######################################
###
### environment setup
###
#######################################

In [55]:
import pandas as pd
import numpy as np
import os
import json

# import matplotlib.pyplot as plt
# from pyproj import Proj, transform, CRS
# from shapely.geometry import Polygon, Point
# import datetime
# from pprint import pprint
# import pprint

# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# display full
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
def show_df(df, head_rows=5):
    print(df.info())
    display(df.head(head_rows))

In [4]:
#######################################
###
### identify source data sets
###
#######################################

In [53]:
files = {
    'via_data': '../data_files/via_datadict.csv',
    'tracts' : '../data_files/bexar_county/Bexar_County_Census_Tracts-shp/Bexar_County_Census_Tracts.shp', 
    'block_groups' : '../data_files/bexar_county/Bexar_County_Census_Block_Groups-shp/Bexar_County_Census_Block_Groups.shp', 
    'blocks' : '../data_files/bexar_county/Bexar_County_Census_Blocks-shp/Bexar_County_Census_Blocks.shp', 
    'adherence_2020' : '../data_files/Adherence_2020.csv',
    'bus_otp_2020' : '../data_files/BusOnTimePerformance_2020.csv',
    'bus_fare_box_2020' : '../data_files/bus_fare_fixed.csv',
    'message_logs_2020' : '../data_files/Logged_Messages.csv',
    'service_miles_2020' : '../data_files/TotalServiceMiles_2020.csv',
    'stops_addr_2020' : '../data_files/Stops_LatLong_GeocodeData_2020.csv',
    'stops_201909' : '../data_files/via_201909/stops.txt', 
    'stops_202004' : '../data_files/via_202004/stops.csv', 
    'trips_201909' : '../data_files/via_201909/trips.txt', 
    'trips_202004' : '../data_files/via_202004/trips.csv', 
    'routes_201909' : '../data_files/via_201909/routes.txt', 
    'routes_202004' : '../data_files/via_202004/routes.csv', 
    'stop_times_201909' : '../data_files/via_201909/stop_times.txt', 
    'stop_times_202004' : '../data_files/via_202004/stop_times.csv', 
    'transfers_201909' : '../data_files/via_201909/transfers.txt', 
    'transfers_202004' : '../data_files/via_202004/transfers.csv', 
    'shapes_201909' : '../data_files/via_201909/shapes.txt', 
    'shapes_202004' : '../data_files/via_202004/shapes.csv', 
    'group_stops' : '../data_files/group_stops-shp',
    'acs5_name':'../data_files/census_data/acs5_name.csv',
    'acs5_key':'../data_files/census_data/acs5_key.csv',
    'census_field_names':'../data_files/census_data/census_field_names.csv',
    'census_groups_names':'../data_files/census_data/census_groups_names.csv',

}

files = {k:v.replace('../','') for (k,v) in files.items()}
files

{'via_data': 'data_files/via_datadict.csv',
 'tracts': 'data_files/bexar_county/Bexar_County_Census_Tracts-shp/Bexar_County_Census_Tracts.shp',
 'block_groups': 'data_files/bexar_county/Bexar_County_Census_Block_Groups-shp/Bexar_County_Census_Block_Groups.shp',
 'blocks': 'data_files/bexar_county/Bexar_County_Census_Blocks-shp/Bexar_County_Census_Blocks.shp',
 'adherence_2020': 'data_files/Adherence_2020.csv',
 'bus_otp_2020': 'data_files/BusOnTimePerformance_2020.csv',
 'bus_fare_box_2020': 'data_files/bus_fare_fixed.csv',
 'message_logs_2020': 'data_files/Logged_Messages.csv',
 'service_miles_2020': 'data_files/TotalServiceMiles_2020.csv',
 'stops_addr_2020': 'data_files/Stops_LatLong_GeocodeData_2020.csv',
 'stops_201909': 'data_files/via_201909/stops.txt',
 'stops_202004': 'data_files/via_202004/stops.csv',
 'trips_201909': 'data_files/via_201909/trips.txt',
 'trips_202004': 'data_files/via_202004/trips.csv',
 'routes_201909': 'data_files/via_201909/routes.txt',
 'routes_202004': '

In [54]:
# files.to_json('files.json')

with open('../data_files/file_dir.json', 'w') as fp:
    json.dump(files, fp, sort_keys=True, indent=4)

In [7]:
#######################################
###
### set up via datathon data
###
#######################################

In [8]:
#######################################
###
### via datathon 2020
###
### adherence
###
#######################################

In [31]:
adh_vehicles = (pd.DataFrame(
    # retype_dataframe(
        pd.Series(
            pd.read_csv(files['adherence_2020'])['VehicleNumber']
            .dropna()
            # .unique()
        )
        .apply(str)
        .sort_values()
        .apply(lambda x: x.replace('.0',''))
        # .astype('string'), columns=['vehicle_id']
        .value_counts()
        .sort_index()
        )        
        .reset_index()
        .rename(columns={'VehicleNumber':'adh_recs','index':'vehicle_id'})
        .set_index('vehicle_id')
        # .drop(columns=['index'])
)

        # .rename(columns={'VehicleNumber':'vehicle_id'}
        # )[['vehicle_id']],
        # gotb_datatype)
        # .vehicle_id.value_counts()
        # ).astype({'VehicleNumber':'string'})
    # df_adh = retype_dataframe(df_adh, gotb_datatype)

show_df(adh_vehicles)

<class 'pandas.core.frame.DataFrame'>
Index: 491 entries, 102 to 987
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   adh_recs  491 non-null    int64
dtypes: int64(1)
memory usage: 7.7+ KB
None


,adh_recs
vehicle_id,
102,174
104,348
105,662
107,283
110,187


In [42]:
# adh_vehicles.tail(5)

In [13]:
#######################################
###
### via datathon 2020
###
### message logs
###
#######################################

In [29]:
# df_logs = pd.read_csv(files['message_logs_2020'])
# df_logs.VehicleNumber = df_logs.VehicleNumber.apply(str)
# show_df(df_logs)

In [30]:
# df_logs.VehicleNumber.value_counts().sort_index()

In [28]:
log_vehicles = (pd.DataFrame(
        pd.Series(
            pd.read_csv(files['message_logs_2020'])['VehicleNumber']
            .dropna()
            # .unique()
        )
        .apply(str)
        .sort_values()
        .apply(lambda x: x.replace('.0',''))
        # .astype('string'), columns=['vehicle_id']
        .value_counts()
        .sort_index()
        )        
        .reset_index()
        .rename(columns={'VehicleNumber':'log_recs','index':'vehicle_id'})
        .set_index('vehicle_id')
        # .drop(columns=['index'])
)

show_df(log_vehicles)

<class 'pandas.core.frame.DataFrame'>
Index: 703 entries, 102 to WTT-0093
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   log_recs  703 non-null    int64
dtypes: int64(1)
memory usage: 11.0+ KB
None


,log_recs
vehicle_id,
102,158
103,189
107,160
108,64
110,130


In [16]:
#######################################
###
### via datathon 2020
###
### service miles
###
#######################################

In [41]:
df_svc_miles = pd.read_csv(files['service_miles_2020'])
show_df(df_svc_miles)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63622 entries, 0 to 63621
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Route                63622 non-null  int64  
 1   RouteName            63622 non-null  object 
 2   Class                62115 non-null  object 
 3   ServiceType          63622 non-null  object 
 4   ScheduledMiles       63622 non-null  float64
 5   AVLRevenueMiles      63622 non-null  float64
 6   AVLDeadHeadMiles     63622 non-null  float64
 7   AVLExemptMiles       63622 non-null  float64
 8   AVLMilesAdjustments  63622 non-null  float64
 9   ServiceDate          63622 non-null  object 
dtypes: float64(5), int64(1), object(4)
memory usage: 4.9+ MB
None


,Route,RouteName,Class,ServiceType,ScheduledMiles,AVLRevenueMiles,AVLDeadHeadMiles,AVLExemptMiles,AVLMilesAdjustments,ServiceDate
0,2,2-BLANCO RD.,Major Radial,SUNDAY,924.35,886.97,46.91,0.0,26.0,1/1/2019
1,3,3-SAN PEDRO SKIP,Major Limited,SUNDAY,707.88,683.66,19.57,0.0,0.0,1/1/2019
2,4,4-SAN PEDRO,Major Radial,SUNDAY,731.48,692.08,34.76,0.0,0.0,1/1/2019
3,5,5-MC CULLOUGH,Major Radial,SUNDAY,484.17,457.37,22.12,0.0,0.0,1/1/2019
4,6,6-US 281 Express,Major Express,SUNDAY,0.00,0.00,0.00,0.0,0.0,1/1/2019


In [18]:
# df_svc_miles.columns


In [33]:
df_vehicles = log_vehicles.join(adh_vehicles, how='outer')
show_df(df_vehicles)

<class 'pandas.core.frame.DataFrame'>
Index: 766 entries, 102 to WTT-0093
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   log_recs  703 non-null    float64
 1   adh_recs  491 non-null    float64
dtypes: float64(2)
memory usage: 18.0+ KB
None


,log_recs,adh_recs
vehicle_id,,
102,158.0,174.0
103,189.0,NaN
104,NaN,348.0
105,NaN,662.0
107,160.0,283.0


In [38]:
len(df_vehicles[df_vehicles.log_recs.isna()])

63

In [39]:
len(df_vehicles[df_vehicles.adh_recs.isna()])

275

In [40]:
# df_vehicles[df_vehicles.log_recs.isna()]

,log_recs,adh_recs
vehicle_id,,
104,NaN,348.0
105,NaN,662.0
111,NaN,137.0
112,NaN,504.0
113,NaN,539.0
114,NaN,247.0
293,NaN,531.0
306,NaN,744.0
310,NaN,678.0
